# 基础数据收集与存储

In [ ]:
!pip install dotenv
# 配置日志 & 加载路径 & 检查Setting清单
import sys
from pathlib import Path
import logging
import pandas as pd
from datetime import datetime

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler()]
)

print("=== 路径设置测试 ===")
print(f"当前工作目录: {Path.cwd()}")
print(f"当前 Python 路径: {sys.path[:3]}...")  # 只显示前3个避免太长

# 正确的路径设置
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

print(f"\n项目根目录: {project_root}")

# 检查系统配置
from src.config.settings import config

print("=== 系统配置 ===")
print(f"特征窗口大小: {config.FEATURE_WINDOW_SIZE}")
print(f"时间窗口设置: {config.TIME_WINDOWS}")
print(f"分类阈值类别数: {len(config.CLASSIFICATION_THRESHOLDS)}")
print()

print("分类标准:")
for class_label, (min_val, max_val) in config.CLASSIFICATION_THRESHOLDS.items():
    print(f"  类别 {class_label}: {min_val}% 到 {max_val}%")

In [ ]:
!pip install pymongo
# 测试数据库连接
from collect.candlestick_handler import mongo_handler

print("正在测试 MongoDB 连接...")

try:
    if mongo_handler.connect():
        print("✓ MongoDB 连接成功")
        
        # 检查现有数据量
        collection = mongo_handler.db[config.MONGODB_COLLECTION]
        record_count = collection.count_documents({})
        print(f"数据库中现有记录数: {record_count:,}")
        
        if record_count > 0:
            # 显示最新几条记录的时间范围
            latest_record = collection.find_one(sort=[("timestamp", -1)])
            earliest_record = collection.find_one(sort=[("timestamp", 1)])
            
            if latest_record and earliest_record:
                latest_time = datetime.fromtimestamp(latest_record['timestamp'] / 1000)
                earliest_time = datetime.fromtimestamp(earliest_record['timestamp'] / 1000)
                print(f"数据时间范围: {earliest_time} 到 {latest_time}")
        
        mongo_handler.close()
    else:
        print("✗ MongoDB 连接失败")
except Exception as e:
    print(f"✗ 数据库连接出错: {e}")

In [ ]:
!pip install requests redis
# 获取最近的市场数据
from src.collect.okex_fetcher import okex_fetcher

print("正在获取最新的行情数据...")

try:
    # 获取最近的 K 线数据
    raw_data = okex_fetcher.fetch_candlesticks(inst_id="ETH-USDT", bar="1H")
    
    if raw_data:
        # 处理数据格式
        processed_data = okex_fetcher._process_candlestick_data(raw_data)
        
        print(f"✓ 成功获取 {len(processed_data)} 条 K 线数据")
        
        # 显示最新几条数据
        df = pd.DataFrame(processed_data[:5])
        print("\n最新 5 条数据:")
        print(df[['inst_id','timestamp', 'open', 'high', 'low', 'close', 'volume']].to_string(index=False))
        
        # 转换时间戳为可读格式
        latest_timestamp = processed_data[0]['timestamp']
        readable_time = datetime.fromtimestamp(latest_timestamp / 1000)
        print(f"\n最新数据时间: {readable_time}")
        
    else:
        print("✗ 未能获取到数据")
        
except Exception as e:
    print(f"✗ 获取数据时出错: {e}")

开始准备模型数据，你可以选择以下操作

### 创建索引
```text
db.candlesticks.createIndex(
    {"inst_id": 1, "timestamp": 1, "bar": 1},
    {
        name: "inst_id_1_timestamp_1_bar_1",
        background: true
    }
);
```

### 清空数据
```text
// Clear all data from candlesticks collection
db.candlesticks.deleteMany({});

// Verify the collection is empty
db.candlesticks.countDocuments();
```

In [ ]:
# 拉取历史数据入库
from src.collect.okex_fetcher import okex_fetcher

# 默认拉10万条，测试的时候可以少拉一些
okex_fetcher.fetch_historical_data(inst_id="ETH-USDT-SWAP", bar="1D", max_records=1000 )

In [ ]:
#测试获取数组格式的数据
from src.collect.mongodb_handler import mongo_handler

mongo_handler.get_candlestick_data(inst_id="ETH-USDT-SWAP", limit=2)